In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'data/{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address
    FROM Customers
    WHERE Country IN ('France', 'Germany') OR City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, COUNT(*) AS NumRecords
    FROM Customers
    GROUP BY Country
    ORDER BY NumRecords DESC
    LIMIT 3
    ''',
    con,
)

,Country,NumRecords
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT s.ShipperName, o.OrderDate
    FROM Orders AS o
    LEFT JOIN Shippers AS s ON s.ShipperID = o.ShipperID
    ORDER BY DATE(substr(o.OrderDate, 7, 4) || '-' ||
                  substr(o.OrderDate, 4, 2) || '-' ||
                  substr(o.OrderDate, 1, 2))
    LIMIT 9,1
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
tasks+=1
# Не у всех товаров цена имеет тип int или float.
# У некторых товаров цена имеет тип string, например, '15.май'.
pd.read_sql(
    '''
    SELECT p.ProductName AS ProductName, CAST(p.Price AS FLOAT) AS Price
    FROM OrderDetails AS od, (
        -- Выбрать самый дорогой заказ.
        SELECT o.OrderID AS OrderID, SUM(CAST(p.Price AS FLOAT)) as MaxPriceOrder
        FROM Orders AS o
        LEFT JOIN OrderDetails AS od ON od.OrderID = o.OrderID
        LEFT JOIN Products AS p ON p.ProductID = od.ProductID
        GROUP BY o.OrderID
        ORDER BY MaxPriceOrder DESC
        LIMIT 1) AS t
    LEFT JOIN Products AS p ON p.ProductID = od.ProductID
    WHERE od.OrderID = t.OrderID
    ''',
    con,
)

,ProductName,Price
0,Rössle Sauerkraut,45.60
1,Thüringer Rostbratwurst,123.79
2,Côte de Blaye,263.50
3,Maxilaku,20.00
4,Tourtière,0.00


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [11]:
tasks+=1
pd.read_sql(
    '''
    -- Самый часто заказываемый товар в каждом заказе.
    SELECT p.ProductName, MAX(od.Quantity) AS MaxQuantity
    FROM OrderDetails AS od
    LEFT JOIN Products AS p ON p.ProductID = od.ProductID
    GROUP BY od.OrderID
    ORDER BY od.OrderID ASC, od.Quantity DESC
    ''',
    con,
)

,ProductName,MaxQuantity
0,Queso Cabrales,12
1,Manjimup Dried Apples,40
2,Manjimup Dried Apples,35
3,Louisiana Fiery Hot Pepper Sauce,20
4,Sir Rodney's Marmalade,40
...,...,...
191,Longlife Tofu,30
192,Sirop d'érable,90
193,Schoggi Schokolade,50
194,Tourtière,80


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT s.SupplierName, s.Country, s.ContactName, s.Phone
    FROM Suppliers AS s, (
        SELECT p.SupplierID AS SupplierID, COUNT(*) AS TotalOrders
        FROM Orders AS o
        LEFT JOIN OrderDetails AS od ON od.OrderID = o.OrderID
        LEFT JOIN Products AS p ON p.ProductID = od.ProductID
        GROUP BY p.SupplierID
        ORDER BY TotalOrders DESC
        LIMIT 5) AS t
    WHERE s.SupplierID = t.SupplierID;
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
1,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
3,Norske Meierier,Norway,Beate Vileid,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT c.Country, c2.CategoryName, SUM(CAST(p.Price AS FLOAT)) AS TotalPrice
    FROM Orders AS o
    LEFT JOIN Customers AS c ON c.CustomerID= o.CustomerID
    LEFT JOIN OrderDetails AS od ON od.OrderID = o.OrderID
    LEFT JOIN Products AS p ON p.ProductID = od.ProductID
    LEFT JOIN Categories c2 ON c2.CategoryID = p.CategoryID
    WHERE c.Country = 'Brazil'
    GROUP BY p.CategoryID
    ORDER BY TotalPrice DESC
    LIMIT 1
    ''',
    con,
)

,Country,CategoryName,TotalPrice
0,Brazil,Beverages,397.5


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [14]:
tasks+=1
pd.read_sql(
    '''
    SELECT (MAX(t.OrderPrice) - MIN(t.OrderPrice)) AS OrderPriceDiff FROM (
        -- Сумма цен в каждом заказе из США.
        SELECT SUM(CAST(p.Price AS FLOAT)) AS OrderPrice
        FROM Orders AS o
        LEFT JOIN Customers AS c ON c.CustomerID= o.CustomerID
        LEFT JOIN OrderDetails AS od ON od.OrderID = o.OrderID
        LEFT JOIN Products AS p ON p.ProductID = od.ProductID
        WHERE c.Country = 'USA'
        GROUP BY o.OrderID HAVING OrderPrice > 0
        ORDER BY OrderPrice DESC
    ) AS t
    ''',
    con,
)

,OrderPriceDiff
0,334.39


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [15]:
tasks+=1
pd.read_sql(
    '''
    SELECT COUNT(*) AS NumOrders, (t.FirstName || ' ' || t.LastName) AS FirstLastName
    FROM Orders AS o, (
        -- Выбрать трех самых молодых сотрудников.
        SELECT EmployeeID, FirstName, LastName
        FROM Employees
        ORDER BY DATE(substr(BirthDate, 7, 4) || '-' ||
                      substr(BirthDate, 4, 2) || '-' ||
                      substr(BirthDate, 1, 2)) DESC
        LIMIT 3) AS t
    WHERE o.EmployeeID = t.EmployeeID
    GROUP BY t.EmployeeID
    ORDER BY NumOrders DESC
    ''',
    con,
)

,NumOrders,FirstLastName
0,31,Janet Leverling
1,29,Nancy Davolio
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT SUM(od.Quantity) AS TotalCrabMeat
    FROM Orders AS o, (
        -- Получить ID для продукта "крабовое мясо"
        SELECT ProductID
        FROM Products AS p
        WHERE LOWER(ProductName) LIKE '%crab%' AND LOWER(ProductName) LIKE '%meat%') AS t
    INNER JOIN OrderDetails AS od ON od.OrderID = o.OrderID AND od.ProductID = t.ProductID
    ''',
    con,
)

,TotalCrabMeat
0,256


In [17]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
